In [48]:
import pandas as pd
from nba_api.stats.endpoints import leaguedashplayerstats, leaguedashteamclutch, leaguegamelog, boxscoreplayertrackv2, playercareerstats, teamplayerdashboard, leaguestandings
from nba_api.stats.static import players, teams
import time
pd.set_option('display.max_columns', None)
from IPython import display as ICD
import warnings
from datetime import date
import pickle
warnings.filterwarnings('ignore')

In [49]:
abbrv_mapping = {'DEN':"1610612743", 
           'SAS':"1610612759", 
           'IND':"1610612754", 
           'MEM':"1610612763", 
           'NOP':"1610612740", 
           'CHI':"1610612741", 
           'MIN':"1610612750", 
           'MIA':"1610612748", 
           'ATL':"1610612737",
           'ORL':"1610612753", 
           'SAC':"1610612758", 
           'UTA':"1610612762", 
           'CHA':"1610612766", 
           'WAS':"1610612764", 
           'HOU':"1610612745", 
           'PHX':"1610612756", 
           'OKC':"1610612760", 
           'POR':"1610612757",
           'LAC':"1610612746", 
           'GSW':"1610612744", 
           'PHI':"1610612755", 
           'LAL':"1610612747", 
           'BKN':"1610612751", 
           'TOR':"1610612761", 
           'BOS':"1610612738", 
           'MIL':"1610612749", 
           'DET':"1610612765",
           'CLE':"1610612739", 
           'NYK':"1610612752",
           'DAL':"1610612742"}

name_mapping = {'Nuggets':"1610612743", 
           'Spurs':"1610612759", 
           'Pacers':"1610612754", 
           'Memphis':"1610612763", 
           'Pelicans':"1610612740", 
           'Bulls':"1610612741", 
           'Timberwolves':"1610612750", 
           'Heat':"1610612748", 
           'Hawks':"1610612737",
           'Magic':"1610612753", 
           'Kings':"1610612758", 
           'Jazz':"1610612762", 
           'Hornets':"1610612766", 
           'Wizards':"1610612764", 
           'Rockets':"1610612745", 
           'Suns':"1610612756", 
           'Thunder':"1610612760", 
           'Trail Blazers':"1610612757",
           'Clippers':"1610612746", 
           'Warriors':"1610612744", 
           '76ers':"1610612755", 
           'Lakers':"1610612747", 
           'Nets':"1610612751", 
           'Raptors':"1610612761", 
           'Celtics':"1610612738", 
           'Bucks':"1610612749", 
           'Pistons':"1610612765",
           'Cavaliers':"1610612739", 
           'Knicks':"1610612752",
           'Mavericks':"1610612742"}

In [50]:
def clean_all_playoff_games(season_name):
    '''
    clean_all_playoff_games - add a few columns and remove irrelevant ones
    '''
    all_games = leaguegamelog.LeagueGameLog(season_type_all_star='Playoffs', season=season_name).get_data_frames()[0]
    all_games['GAME_DATE'] = pd.to_datetime(all_games['GAME_DATE']) 
    all_games = all_games.sort_values('GAME_DATE',ascending = False)
    all_games['OpponentTeamID'] = all_games['MATCHUP'].apply(lambda x: int(abbrv_mapping[x[-3:]]))
    all_games['OPTS'] = all_games['PTS'] - all_games['PLUS_MINUS']
    all_games = all_games[['GAME_ID','GAME_DATE','TEAM_ID','OpponentTeamID','MATCHUP','PTS','OPTS']]
    return all_games.rename(columns = {'GAME_ID':'GameID','GAME_DATE':'GameDate', 'TEAM_ID':'TeamID'})

def clean_all_games(season_name):
    '''
    clean_all_playoff_games - add a few columns and remove irrelevant ones
    '''
    all_games = leaguegamelog.LeagueGameLog(season=season_name).get_data_frames()[0]
    all_games['GAME_DATE'] = pd.to_datetime(all_games['GAME_DATE']) 
    all_games = all_games.sort_values('GAME_DATE',ascending = False)
    all_games['OpponentTeamID'] = all_games['MATCHUP'].apply(lambda x: int(abbrv_mapping[x[-3:]]))
    all_games['OPTS'] = all_games['PTS'] - all_games['PLUS_MINUS']
    all_games = all_games[['GAME_ID','GAME_DATE','TEAM_ID','OpponentTeamID','MATCHUP','PTS','OPTS']]
    return all_games.rename(columns = {'GAME_ID':'GameID','GAME_DATE':'GameDate', 'TEAM_ID':'TeamID'})

all_playoff_games = clean_all_playoff_games('2019-20')
all_regseason_playoff_games = clean_all_games('2019-20')
all_reg_season_playoff_games = pd.concat([all_playoff_games, all_regseason_playoff_games]).sort_values('GameDate',ascending = False)
all_reg_season_playoff_games

,GameID,GameDate,TeamID,OpponentTeamID,MATCHUP,PTS,OPTS
91,0041900177,2020-09-02,1610612745,1610612760,HOU vs. OKC,104,102
89,0041900202,2020-09-02,1610612748,1610612749,MIA @ MIL,116,114
88,0041900202,2020-09-02,1610612749,1610612748,MIL vs. MIA,114,116
90,0041900177,2020-09-02,1610612760,1610612745,OKC @ HOU,102,104
87,0041900167,2020-09-01,1610612743,1610612762,DEN vs. UTA,80,78
...,...,...,...,...,...,...,...
15,0021900007,2019-10-23,1610612763,1610612748,MEM @ MIA,101,120
2,0021900001,2019-10-22,1610612761,1610612740,TOR vs. NOP,130,122
1,0021900002,2019-10-22,1610612747,1610612746,LAL @ LAC,102,112
3,0021900001,2019-10-22,1610612740,1610612761,NOP @ TOR,122,130


In [51]:
player_rotation = pd.read_csv('players_rotation.csv')
players_df = pd.read_csv('players.csv')
first_game_missing_mins = player_rotation.groupby('TEAM_ID').sum()['MIN'] / players_df.groupby('TEAM_ID').sum()['MIN']

In [52]:
def missing_mins(curr_game_id, team_id, season_name):
    '''
    missing_mins: gets the fraction of the season's minutes played by injured/inactive/traded players
    '''
    time.sleep(0.5)
    # get the season stats of the team
    season_stats = teamplayerdashboard.TeamPlayerDashboard(team_id, season=season_name).get_data_frames()[1]
    
    # get the game currently being assessed
    curr_game = boxscoreplayertrackv2.BoxScorePlayerTrackV2(game_id=curr_game_id).get_data_frames()[0]
    
    player_game_stats = curr_game[curr_game['TEAM_ID'] == int(team_id)]
    combined = pd.merge(season_stats, player_game_stats, how='left', on='PLAYER_ID')
    
    total_mins = combined['MIN_x'].sum()
    missing_mins = combined[(combined['MIN_y'] == "0:00") | (combined['MIN_y'].isna())]['MIN_x'].sum()
    return (total_mins - missing_mins) / total_mins

In [53]:
games = pd.read_csv('predictions/games.csv')
games['Date'] = games['Date'].astype('datetime64')
games

,Date,GameID,TeamID,OpponentTeamID,TeamName,OpponentTeamName,TeamWinProba,OpponentTeamWinProba,TeamPoints,OpponentTeamPoints,TeamPointsActual,OpponentTeamPointsActual,GameNum,Unnamed: 13,Unnamed: 14
0,2020-08-17,41900161.0,1.610613e+09,1.610613e+09,Nuggets,Jazz,0.446276,0.553724,104.907378,106.258136,135.0,125.0,1.0,NaN,NaN
1,2020-08-17,41900111.0,1.610613e+09,1.610613e+09,Raptors,Nets,0.587735,0.412265,107.809764,105.592550,134.0,110.0,1.0,NaN,NaN
2,2020-08-17,41900121.0,1.610613e+09,1.610613e+09,Celtics,76ers,0.390522,0.609478,100.521847,103.301425,109.0,101.0,1.0,NaN,NaN
3,2020-08-17,41900151.0,1.610613e+09,1.610613e+09,Clippers,Mavericks,0.676566,0.323434,107.580625,102.999441,118.0,110.0,1.0,NaN,NaN
4,2020-08-18,41900101.0,1.610613e+09,1.610613e+09,Bucks,Magic,0.616207,0.383793,107.926797,104.971470,110.0,122.0,1.0,NaN,NaN
5,2020-08-18,41900131.0,1.610613e+09,1.610613e+09,Pacers,Heat,0.690723,0.309277,105.559310,100.580296,101.0,113.0,1.0,NaN,NaN
6,2020-08-18,41900171.0,1.610613e+09,1.610613e+09,Rockets,Thunder,0.493277,0.506723,104.115003,104.283525,123.0,108.0,1.0,NaN,NaN
7,2020-08-18,41900141.0,1.610613e+09,1.610613e+09,Lakers,Trail Blazers,0.581614,0.418386,108.093849,106.033605,93.0,100.0,1.0,NaN,NaN
8,2020-08-19,41900162.0,1.610613e+09,1.610613e+09,Nuggets,Jazz,0.683257,0.316743,111.777117,107.008869,105.0,124.0,2.0,NaN,NaN
9,2020-08-19,41900112.0,1.610613e+09,1.610613e+09,Raptors,Nets,0.927164,0.072836,118.184497,103.634586,104.0,99.0,2.0,NaN,NaN


In [54]:
def get_gameid(row):
    if(not pd.isna(row['GameID'])):
        return row['GameID']
    temp = all_playoff_games[all_playoff_games['TeamID'] == row['TeamID']]
    temp = all_playoff_games[all_playoff_games['OpponentTeamID'] == row['OpponentTeamID']]
    temp = temp[temp['GameDate'].apply(lambda x: x.date() == row['Date'].date())]
    if(len(temp) != 0):
        return temp['GameID'].iloc[0]
    return row['GameID']
def get_game_info(row, attribute, alt_attribute):
    if(pd.isna(row['GameID'])):
        return row[alt_attribute]
    try:
        return all_playoff_games[(all_playoff_games['GameID'] == row['GameID']) & (all_playoff_games['TeamID'] == row['TeamID'])][attribute].iloc[0]
    except:
        return row[alt_attribute]
def get_game_num(row):
    if(not pd.isna(row['GameNum'])):
        return row['GameNum']
    temp = all_playoff_games[all_playoff_games['TeamID'] == row['TeamID']]
    temp = temp[temp['OpponentTeamID'] == row['OpponentTeamID']]
    return len(temp) + 1

games['TeamID'] = games['TeamName'].apply(lambda x: int(name_mapping[x]))
games['OpponentTeamID'] = games['OpponentTeamName'].apply(lambda x: int(name_mapping[x]))
games['GameID'] = games.apply(lambda x: get_gameid(x), axis = 1)
games['Date'] = games.apply(lambda x: get_game_info(x, 'GameDate', 'Date'), axis = 1)
games['TeamPointsActual'] = games.apply(lambda x: get_game_info(x, 'PTS', 'TeamPointsActual'), axis = 1)
games['OpponentTeamPointsActual'] = games.apply(lambda x: get_game_info(x, 'OPTS', 'OpponentTeamPointsActual'), axis = 1)
games['GameNum'] = games.apply(lambda x: get_game_num(x), axis = 1)
games

,Date,GameID,TeamID,OpponentTeamID,TeamName,OpponentTeamName,TeamWinProba,OpponentTeamWinProba,TeamPoints,OpponentTeamPoints,TeamPointsActual,OpponentTeamPointsActual,GameNum,Unnamed: 13,Unnamed: 14
0,2020-08-17,4.19002e+07,1610612743,1610612762,Nuggets,Jazz,0.446276,0.553724,104.907378,106.258136,135.0,125.0,1.0,NaN,NaN
1,2020-08-17,4.19001e+07,1610612761,1610612751,Raptors,Nets,0.587735,0.412265,107.809764,105.592550,134.0,110.0,1.0,NaN,NaN
2,2020-08-17,4.19001e+07,1610612738,1610612755,Celtics,76ers,0.390522,0.609478,100.521847,103.301425,109.0,101.0,1.0,NaN,NaN
3,2020-08-17,4.19002e+07,1610612746,1610612742,Clippers,Mavericks,0.676566,0.323434,107.580625,102.999441,118.0,110.0,1.0,NaN,NaN
4,2020-08-18,4.19001e+07,1610612749,1610612753,Bucks,Magic,0.616207,0.383793,107.926797,104.971470,110.0,122.0,1.0,NaN,NaN
5,2020-08-18,4.19001e+07,1610612754,1610612748,Pacers,Heat,0.690723,0.309277,105.559310,100.580296,101.0,113.0,1.0,NaN,NaN
6,2020-08-18,4.19002e+07,1610612745,1610612760,Rockets,Thunder,0.493277,0.506723,104.115003,104.283525,123.0,108.0,1.0,NaN,NaN
7,2020-08-18,4.19001e+07,1610612747,1610612757,Lakers,Trail Blazers,0.581614,0.418386,108.093849,106.033605,93.0,100.0,1.0,NaN,NaN
8,2020-08-19,4.19002e+07,1610612743,1610612762,Nuggets,Jazz,0.683257,0.316743,111.777117,107.008869,105.0,124.0,2.0,NaN,NaN
9,2020-08-19,4.19001e+07,1610612761,1610612751,Raptors,Nets,0.927164,0.072836,118.184497,103.634586,104.0,99.0,2.0,NaN,NaN


In [55]:
def get_series_lead(games, curr_game):
    '''
    gets_series_length - get the game number of the series
    '''
    series = games[(games['TeamID'] == curr_game['TeamID']) & (games['OpponentTeamID'] == curr_game['OpponentTeamID'])]
    len_series = len(series)
    games_won = len(series[series['PTS'] > series['OPTS']])
    return 2 * games_won - len_series

def get_series_win_percent(games, curr_game):
    '''
    gets_series_length - get the game number of the series
    '''
    series = games[(games['TeamID'] == curr_game['TeamID']) & (games['OpponentTeamID'] == curr_game['OpponentTeamID'])]
    len_series = len(series) + 1
    games_won = len(series[series['PTS'] > series['OPTS']])
    return games_won/len_series

In [56]:
# creating the games dataframe
games_copy = games[pd.isna(games['TeamWinProba'])][['Date','TeamID', 'OpponentTeamID']]
games_copy_reversed = games_copy.copy()
games_copy_reversed['TeamID'] = games_copy['OpponentTeamID']
games_copy_reversed['OpponentTeamID'] = games_copy['TeamID']

all_games = pd.concat([games_copy, games_copy_reversed]).sort_index().reset_index(drop=True)

reg_season_ratings = leaguestandings.LeagueStandings(season='2019-20', season_type = 'Regular Season').get_data_frames()[0]
reg_season_ratings = reg_season_ratings[['TeamID','PointsPG','OppPointsPG']]

#ratings
all_games = pd.merge(all_games, reg_season_ratings, how='left', on='TeamID')
all_games = pd.merge(all_games, reg_season_ratings, how='left', left_on='OpponentTeamID', right_on='TeamID', suffixes=['','Opponent'])

all_games['SeriesGP'] = all_games.apply(lambda x: 1 + len(all_playoff_games[(all_playoff_games['TeamID'] == x['OpponentTeamID']) & (all_playoff_games['OpponentTeamID'] == x['TeamID'])]), axis = 1) 
all_games['SeriesLead'] = all_games.apply(lambda x: get_series_lead(all_playoff_games, x), axis = 1) 
all_games['SeriesWinPercent'] = all_games.apply(lambda x: get_series_win_percent(all_playoff_games, x), axis = 1) 





all_games['PrevPTS'] = all_games.apply(lambda x: all_playoff_games[all_playoff_games['TeamID'] == x['TeamID']]['PTS'].iloc[0], axis = 1) 
all_games['PrevOPTS'] = all_games.apply(lambda x: all_playoff_games[all_playoff_games['TeamID'] == x['TeamID']]['OPTS'].iloc[0], axis = 1) 
all_games['PrevMargin'] = all_games['PrevPTS'] - all_games['PrevOPTS']


all_games['MissingMins'] = all_games.apply(lambda x: missing_mins(all_playoff_games[all_playoff_games['TeamID'] == x['TeamID']]['GameID'].iloc[0], x['TeamID'], '2019-20'), axis = 1)
# first_game_missing_mins_copy = first_game_missing_mins.to_frame().reset_index()
# first_game_missing_mins_copy.columns = ['TeamID', 'MissingMins']
# all_games = pd.merge(all_games, first_game_missing_mins_copy, how='left', on='TeamID')


playoff_experience = pd.read_csv('playoff_experience.csv')
all_games = pd.merge(all_games, playoff_experience, how = 'left', on = 'TeamID')
all_games


,Date,TeamID,OpponentTeamID,PointsPG,OppPointsPG,TeamIDOpponent,PointsPGOpponent,OppPointsPGOpponent,SeriesGP,SeriesLead,SeriesWinPercent,PrevPTS,PrevOPTS,PrevMargin,MissingMins,Age,PlayoffMins
0,2020-09-03,1610612761,1610612738,112.8,106.5,1610612738,113.7,107.3,3,-2,0.000000,99,102,-3,0.825932,397.0,13204.620000
1,2020-09-03,1610612738,1610612761,113.7,107.3,1610612761,112.8,106.5,3,2,0.666667,102,99,3,0.759453,379.0,5772.186667
2,2020-09-03,1610612746,1610612743,116.3,109.9,1610612743,111.3,109.2,1,0,0.000000,111,97,14,0.763382,412.0,13322.215000
3,2020-09-03,1610612743,1610612746,111.3,109.2,1610612746,116.3,109.9,1,0,0.000000,80,78,2,0.778814,391.0,6895.763333
4,2020-09-04,1610612749,1610612748,118.7,108.6,1610612748,112.0,109.1,3,-2,0.000000,114,116,-2,0.806116,446.0,16320.056667
5,2020-09-04,1610612748,1610612749,112.0,109.1,1610612749,118.7,108.6,3,2,0.666667,116,114,2,0.863442,398.0,10360.451667
6,2020-09-04,1610612747,1610612745,113.4,107.6,1610612745,117.8,114.8,1,0,0.000000,131,122,9,0.829908,454.0,29173.070000
7,2020-09-04,1610612745,1610612747,117.8,114.8,1610612747,113.4,107.6,1,0,0.000000,104,102,2,0.728101,434.0,16812.746667


In [57]:
# bias boosted monte carlo simulation

In [58]:
import random
import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import LinearRegression
def get_weight(x, col_value, range_val):
    return 1/(00.1+abs((x-col_value)/range_val))
def get_rand_sample(df, col_name, col_value, sample_col_name, N, range_val):
    weights = df[col_name].apply(lambda x: get_weight(x,col_value, range_val))
    to_return = random.choices(population=list(df[sample_col_name]), 
                               weights=weights,
                               k=N)
    return to_return

In [59]:
pts_model = pickle.load( open( "pts_model", "rb" ) )
opts_model = pickle.load( open( "opts_model", "rb" ) )
range_vals = pickle.load( open( "range_vals", "rb" ) )
playoff_data = pd.read_csv('all_playoff_games.csv')
offense_cols = ['PlayoffMins','Age','SeriesGP','SeriesLead', 'SeriesWinPercent', 'MissingMins','PrevMargin','PointsPG','OppPointsPGOpponent','PrevPTS']
defense_cols = ['PlayoffMins','Age','SeriesGP','SeriesLead', 'SeriesWinPercent', 'MissingMins','PrevMargin','OppPointsPG','PointsPGOpponent','PrevOPTS']

In [60]:
def get_predictions(playoff_games, cols, predictor, N, model = None):
    predictions = playoff_games[['Date', 'TeamID','OpponentTeamID']]
    col_sum = []
    for x in cols:
        col_sum.append(x + "Sum")
    for col in cols:
        predictions[col] = playoff_games.apply(lambda x: get_rand_sample(playoff_data,col,x[col],predictor,N,range_vals[col]), axis = 1)
    for col in range(len(cols)):
        predictions[col_sum[col]] = predictions[cols[col]].apply(lambda x: sum(x)/N)
    X = predictions[col_sum]
    
    coefs = pd.Series(model.coef_, index = col_sum).abs()
    # predictions[predictor + 'Combined'] = predictions[cols].apply(lambda x: (np.array(x.values.tolist()) * coefs.values[:, None]).sum(axis=0)/coefs.sum(), axis = 1)
    predictions[predictor + 'Combined'] = predictions[cols].apply(lambda x: model.predict(np.array(x.values.tolist()).transpose()), axis = 1)
    predictions[predictor + 'LinearRegression'] = model.predict(X)
    temp = predictions[['TeamID'] + col_sum + [predictor+'LinearRegression']]
    temp['Name'] = temp['TeamID'].apply(lambda x: list(name_mapping.keys())[list(pd.Series(list(name_mapping.values())).astype(int)).index(x)])
    ICD.display(temp)
    temp.to_csv('predictions_intermed/' + predictor+date.today().strftime('%d-%m-%Y') + '.csv', index = False)
    return predictions[[predictor + 'Combined', predictor + 'LinearRegression']]

In [61]:
pts_df = get_predictions(all_games, offense_cols, 'AdjustedPTS', 10000, pts_model)
opts_df = get_predictions(all_games, defense_cols, 'AdjustedOPTS', 10000, opts_model)
final = pd.concat([all_games, pts_df, opts_df], axis = 1)
final['WinProba'] = final.apply(lambda x: np.count_nonzero(x['AdjustedPTSCombined'] > x['AdjustedOPTSCombined'])/1000, axis = 1)
final['WinProbaScaled'] = final.apply(lambda x: x['WinProba']* 1/final[((final['SeriesGP'] == x['SeriesGP'])) & (((final['TeamID'] == x['TeamID']) & (final['OpponentTeamID'] == x['OpponentTeamID'])) | ((final['TeamID'] == x['OpponentTeamID']) & (final['OpponentTeamID'] == x['TeamID'])))]['WinProba'].sum(), axis = 1)
final['WinProba2x'] = final.apply(lambda x: max(0,min(1,2*x['WinProbaScaled']-0.5)), axis = 1)
ICD.display(final)
final = final[['Date','TeamID','OpponentTeamID','AdjustedPTSLinearRegression','AdjustedOPTSLinearRegression', 'WinProba2x']]

,TeamID,PlayoffMinsSum,AgeSum,SeriesGPSum,SeriesLeadSum,SeriesWinPercentSum,MissingMinsSum,PrevMarginSum,PointsPGSum,OppPointsPGOpponentSum,PrevPTSSum,AdjustedPTSLinearRegression,Name
0,1610612761,105.433292,106.094209,107.406958,105.600301,105.961755,106.057473,105.831307,106.720887,106.198222,104.988334,106.901941,Raptors
1,1610612738,105.219877,106.071131,107.032120,106.860384,106.653992,105.882904,105.985034,107.353786,106.020418,105.612077,108.049903,Celtics
2,1610612746,105.502668,106.042726,106.213091,105.710024,106.061999,105.692927,106.993922,108.540624,106.024689,107.093930,111.099913,Clippers
3,1610612743,105.122608,106.029512,106.488921,105.766883,105.805436,105.924477,106.181048,106.517800,106.311410,104.069608,103.469045,Nuggets
4,1610612749,105.633688,106.037168,107.159569,105.499235,106.034722,105.984067,105.700309,108.339671,106.036980,107.368322,111.817190,Bucks
5,1610612748,104.880252,106.031622,107.321792,107.012540,106.731663,106.257097,106.281927,106.552289,105.968724,107.348359,110.514570,Heat
6,1610612747,107.805432,106.255757,106.338320,105.601379,106.006337,106.257209,106.422536,107.195406,107.576039,107.441089,115.500774,Lakers
7,1610612745,105.973011,106.132594,106.198374,105.510068,105.940841,105.691618,105.998836,108.808478,105.908623,105.829697,108.418162,Rockets


,TeamID,PlayoffMinsSum,AgeSum,SeriesGPSum,SeriesLeadSum,SeriesWinPercentSum,MissingMinsSum,PrevMarginSum,OppPointsPGSum,PointsPGOpponentSum,PrevOPTSSum,AdjustedOPTSLinearRegression,Name
0,1610612761,105.603075,106.183792,107.051549,106.906616,106.958389,105.827135,106.228895,105.914827,107.506298,105.588295,109.481708,Raptors
1,1610612738,106.525819,105.909675,107.018902,105.647704,105.585914,105.765701,105.901687,106.053729,106.853106,105.231174,107.597391,Celtics
2,1610612746,105.575234,106.150506,106.259167,105.693024,107.226572,105.321411,105.923044,106.364886,106.415171,105.125418,104.522996,Clippers
3,1610612743,106.267219,106.033107,106.074815,105.373049,107.180720,105.506942,105.740155,106.127206,108.531869,104.211156,107.321923,Nuggets
4,1610612749,105.343422,106.163014,107.273065,106.982644,107.229829,105.351890,106.272612,106.087583,106.705481,107.295442,109.676834,Bucks
5,1610612748,106.020599,105.960479,107.104499,105.363860,105.665414,106.034967,105.865539,106.055788,108.666962,107.105646,113.093417,Heat
6,1610612747,105.927663,106.114598,106.210309,105.657398,107.199941,105.516985,105.749978,105.930749,108.715789,107.634785,111.594440,Lakers
7,1610612745,105.366140,106.139359,106.017402,105.676079,106.732633,106.046947,105.844029,107.277728,107.079623,105.431148,108.804752,Rockets


,Date,TeamID,OpponentTeamID,PointsPG,OppPointsPG,TeamIDOpponent,PointsPGOpponent,OppPointsPGOpponent,SeriesGP,SeriesLead,SeriesWinPercent,PrevPTS,PrevOPTS,PrevMargin,MissingMins,Age,PlayoffMins,AdjustedPTSCombined,AdjustedPTSLinearRegression,AdjustedOPTSCombined,AdjustedOPTSLinearRegression,WinProba,WinProbaScaled,WinProba2x
0,2020-09-03,1610612761,1610612738,112.8,106.5,1610612738,113.7,107.3,3,-2,0.000000,99,102,-3,0.825932,397.0,13204.620000,"[209.3293386847963, 85.25201048263943, 79.4933...",106.901941,"[97.19177480765256, 152.96046640454347, 123.60...",109.481708,4.877,0.491584,0.483167
1,2020-09-03,1610612738,1610612761,113.7,107.3,1610612761,112.8,106.5,3,2,0.666667,102,99,3,0.759453,379.0,5772.186667,"[124.58373664974624, 82.27130018603384, 151.01...",108.049903,"[103.94507003416561, 106.2897182264694, 122.20...",107.597391,5.044,0.508416,0.516833
2,2020-09-03,1610612746,1610612743,116.3,109.9,1610612743,111.3,109.2,1,0,0.000000,111,97,14,0.763382,412.0,13322.215000,"[89.41217907451892, 115.48885514482117, 86.233...",111.099913,"[230.6053608325817, 155.0123268490138, 107.462...",104.522996,5.345,0.526083,0.552165
3,2020-09-03,1610612743,1610612746,111.3,109.2,1610612746,116.3,109.9,1,0,0.000000,80,78,2,0.778814,391.0,6895.763333,"[95.13079364664668, 167.05018796071386, 109.03...",103.469045,"[104.23962777328688, 104.64742927612474, 181.3...",107.321923,4.815,0.473917,0.447835
4,2020-09-04,1610612749,1610612748,118.7,108.6,1610612748,112.0,109.1,3,-2,0.000000,114,116,-2,0.806116,446.0,16320.056667,"[137.84151160537272, 60.23230694498284, 66.123...",111.817190,"[134.33674575792543, 48.56877583372602, 103.53...",109.676834,5.149,0.516708,0.533417
5,2020-09-04,1610612748,1610612749,112.0,109.1,1610612749,118.7,108.6,3,2,0.666667,116,114,2,0.863442,398.0,10360.451667,"[165.2823181682602, 64.11928506961749, 148.995...",110.514570,"[116.21044526910305, 153.67012768066274, 127.1...",113.093417,4.816,0.483292,0.466583
6,2020-09-04,1610612747,1610612745,113.4,107.6,1610612745,117.8,114.8,1,0,0.000000,131,122,9,0.829908,454.0,29173.070000,"[182.7877540905099, 123.01580446581374, 55.885...",115.500774,"[193.9331384281479, 117.37909992759046, 143.78...",111.594440,5.179,0.507944,0.515889
7,2020-09-04,1610612745,1610612747,117.8,114.8,1610612747,113.4,107.6,1,0,0.000000,104,102,2,0.728101,434.0,16812.746667,"[218.28452071419474, 66.70683698900552, 112.09...",108.418162,"[140.66794716146865, 144.90452944162303, 134.5...",108.804752,5.017,0.492056,0.484111


In [62]:
for x in range(1, len(final),2):
    temp = final['AdjustedPTSLinearRegression'].iloc[x]
    final['AdjustedPTSLinearRegression'].iloc[x] = final['AdjustedOPTSLinearRegression'].iloc[x]
    final['AdjustedOPTSLinearRegression'].iloc[x] = temp
final =final.groupby(np.arange(len(final))//2).agg({'Date': 'first',
                                            'TeamID':'first',
                                             'OpponentTeamID':'first',
                                             'AdjustedPTSLinearRegression':'mean',
                                             'AdjustedOPTSLinearRegression':'mean',
                                             'WinProba2x':'first'})

In [63]:
import scipy.stats
norm_dist = scipy.stats.norm(0, 10)
final['ProbaNorm'] = final.apply(lambda x: norm_dist.cdf(x['AdjustedPTSLinearRegression'] - x['AdjustedOPTSLinearRegression']), axis = 1)
final

,Date,TeamID,OpponentTeamID,AdjustedPTSLinearRegression,AdjustedOPTSLinearRegression,WinProba2x,ProbaNorm
0,2020-09-03,1610612761,1610612738,107.249666,108.765806,0.483167,0.439746
1,2020-09-03,1610612746,1610612743,109.210918,103.996020,0.552165,0.698987
2,2020-09-04,1610612749,1610612748,112.455304,110.095702,0.533417,0.593268
3,2020-09-04,1610612747,1610612745,112.152763,110.006301,0.515889,0.584978


In [64]:
games

,Date,GameID,TeamID,OpponentTeamID,TeamName,OpponentTeamName,TeamWinProba,OpponentTeamWinProba,TeamPoints,OpponentTeamPoints,TeamPointsActual,OpponentTeamPointsActual,GameNum,Unnamed: 13,Unnamed: 14
0,2020-08-17,4.19002e+07,1610612743,1610612762,Nuggets,Jazz,0.446276,0.553724,104.907378,106.258136,135.0,125.0,1.0,NaN,NaN
1,2020-08-17,4.19001e+07,1610612761,1610612751,Raptors,Nets,0.587735,0.412265,107.809764,105.592550,134.0,110.0,1.0,NaN,NaN
2,2020-08-17,4.19001e+07,1610612738,1610612755,Celtics,76ers,0.390522,0.609478,100.521847,103.301425,109.0,101.0,1.0,NaN,NaN
3,2020-08-17,4.19002e+07,1610612746,1610612742,Clippers,Mavericks,0.676566,0.323434,107.580625,102.999441,118.0,110.0,1.0,NaN,NaN
4,2020-08-18,4.19001e+07,1610612749,1610612753,Bucks,Magic,0.616207,0.383793,107.926797,104.971470,110.0,122.0,1.0,NaN,NaN
5,2020-08-18,4.19001e+07,1610612754,1610612748,Pacers,Heat,0.690723,0.309277,105.559310,100.580296,101.0,113.0,1.0,NaN,NaN
6,2020-08-18,4.19002e+07,1610612745,1610612760,Rockets,Thunder,0.493277,0.506723,104.115003,104.283525,123.0,108.0,1.0,NaN,NaN
7,2020-08-18,4.19001e+07,1610612747,1610612757,Lakers,Trail Blazers,0.581614,0.418386,108.093849,106.033605,93.0,100.0,1.0,NaN,NaN
8,2020-08-19,4.19002e+07,1610612743,1610612762,Nuggets,Jazz,0.683257,0.316743,111.777117,107.008869,105.0,124.0,2.0,NaN,NaN
9,2020-08-19,4.19001e+07,1610612761,1610612751,Raptors,Nets,0.927164,0.072836,118.184497,103.634586,104.0,99.0,2.0,NaN,NaN


In [65]:
def get_game_info_calculated(row, final_col, games_col):
    if(not pd.isna(row[games_col])):
        return row[games_col]
    temp = final[final['TeamID'] == row['TeamID']]
    temp = final[final['OpponentTeamID'] == row['OpponentTeamID']]
    return temp[final_col].iloc[0]

games['TeamWinProba'] = games.apply(lambda x: get_game_info_calculated(x, 'ProbaNorm', 'TeamWinProba'), axis = 1)
games['OpponentTeamWinProba'] = games.apply(lambda x: 1 - x['TeamWinProba'], axis = 1)
games['TeamPoints'] = games.apply(lambda x: get_game_info_calculated(x, 'AdjustedPTSLinearRegression', 'TeamPoints'), axis = 1)
games['OpponentTeamPoints'] = games.apply(lambda x: get_game_info_calculated(x, 'AdjustedOPTSLinearRegression', 'OpponentTeamPoints'), axis = 1)
games

,Date,GameID,TeamID,OpponentTeamID,TeamName,OpponentTeamName,TeamWinProba,OpponentTeamWinProba,TeamPoints,OpponentTeamPoints,TeamPointsActual,OpponentTeamPointsActual,GameNum,Unnamed: 13,Unnamed: 14
0,2020-08-17,4.19002e+07,1610612743,1610612762,Nuggets,Jazz,0.446276,0.553724,104.907378,106.258136,135.0,125.0,1.0,NaN,NaN
1,2020-08-17,4.19001e+07,1610612761,1610612751,Raptors,Nets,0.587735,0.412265,107.809764,105.592550,134.0,110.0,1.0,NaN,NaN
2,2020-08-17,4.19001e+07,1610612738,1610612755,Celtics,76ers,0.390522,0.609478,100.521847,103.301425,109.0,101.0,1.0,NaN,NaN
3,2020-08-17,4.19002e+07,1610612746,1610612742,Clippers,Mavericks,0.676566,0.323434,107.580625,102.999441,118.0,110.0,1.0,NaN,NaN
4,2020-08-18,4.19001e+07,1610612749,1610612753,Bucks,Magic,0.616207,0.383793,107.926797,104.971470,110.0,122.0,1.0,NaN,NaN
5,2020-08-18,4.19001e+07,1610612754,1610612748,Pacers,Heat,0.690723,0.309277,105.559310,100.580296,101.0,113.0,1.0,NaN,NaN
6,2020-08-18,4.19002e+07,1610612745,1610612760,Rockets,Thunder,0.493277,0.506723,104.115003,104.283525,123.0,108.0,1.0,NaN,NaN
7,2020-08-18,4.19001e+07,1610612747,1610612757,Lakers,Trail Blazers,0.581614,0.418386,108.093849,106.033605,93.0,100.0,1.0,NaN,NaN
8,2020-08-19,4.19002e+07,1610612743,1610612762,Nuggets,Jazz,0.683257,0.316743,111.777117,107.008869,105.0,124.0,2.0,NaN,NaN
9,2020-08-19,4.19001e+07,1610612761,1610612751,Raptors,Nets,0.927164,0.072836,118.184497,103.634586,104.0,99.0,2.0,NaN,NaN


In [66]:
today = date.today()

pd.read_csv('predictions/games.csv').to_csv('predictions/games_' + today.strftime("%d-%m-%Y") +'_before.csv', index = False)
games.to_csv('predictions/games.csv', index = False)
games.to_csv('predictions/games_' + today.strftime("%d-%m-%Y") +'_after.csv', index = False)